<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/Web_scraping_Accidents_plus_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install beautifulsoup4
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 1.4MB/s 


In [0]:
import sys
from datetime import *
from time import *

def get_yobi(inDate):
  
  yobi = ["月","火","水","木","金","土","日"]

  a = datetime.strptime(inDate,'%Y/%m/%d')

  #return yobi[a.weekday()]
  return a.weekday()

In [0]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import pandas as pd
#import MeCab
from google.colab import files

#https://accident.laboneko.jp/accidents?limit=10&offset=0
#https://accident.laboneko.jp/accidents?limit=11680&offset=0

# 親ページURL
url_pr = 'https://accident.laboneko.jp/accidents?limit=11680&offset=0'

# データフレームを定義
columns = ["date","time","yobi","line","from_st","to_st","st"]
df = pd.DataFrame(columns=columns)

html_doc = urlopen(url_pr)

# BeautifulSoupでスクレイピング
soup = BeautifulSoup(html_doc, 'html.parser')
# ラッピング単位のクラスを取得
table  = soup.find_all("table", {"class": "table table-striped"})[0]
rows = table.findAll("tr")

for row in rows:
  dataRow = []
  for cell in row.findAll("td"):
    dataRow.append(cell.text)

  #listに何か入っている場合
  if dataRow:

    v_date = dataRow[0][0:10]
    v_time = dataRow[0][11:16]
    v_yobi = get_yobi(v_date)

    v_line = dataRow[1]

    v_st_info = dataRow[2]
    v_st_info_sp = v_st_info.split("〜")

    if len(v_st_info_sp) >= 2:
      v_from_st = v_st_info_sp[0]
      v_to_st = v_st_info_sp[1]
      v_st = v_st_info_sp[0]
    else:
      v_from_st = v_st_info
      v_to_st = v_st_info
      v_st = v_st_info

    # 各データをデータフレームに格納
    se = pd.Series([v_date,v_time,v_yobi,v_line,v_from_st,v_to_st,v_st], columns)
    df = df.append(se, columns)

# 収集したデータをCSV形式で保存
filename = "Accident_list.csv"
df.to_csv(filename, encoding = 'utf-8',index=False)
#files.download(filename)


##ファイルがあればここから実行

In [2]:
import pandas as pd

df_ac = pd.read_csv("Accident_list.csv")

#鉄道会社の人身事故発生順にソート
df_line = df_ac["line"].value_counts().reset_index()

#SeriseをDataFrameに変換
col_name = ["line","count"]
df_line = pd.DataFrame(df_line)

df_line.columns = col_name
df_line.head()

,line,count
0,東武東上線,291
1,山陽本線_(JR西日本),263
2,京浜東北線,245
3,中央快速線,244
4,東海道線_(JR東海),244


In [0]:
#件数は削除
df_line = df_line.drop("count",axis=1)
df_line

#CSVに出力
df_line.to_csv("line_List_Candidate.csv", encoding = 'utf-8',index=False,header=False)

In [4]:
import itertools

kanto_line = pd.read_table('kanto_line.txt', header=None)

#dataframeをlist化
kanto_line_list = kanto_line.values.tolist()
#2次元listから1次元listへ
kanto_line_list = list(itertools.chain.from_iterable(kanto_line_list))

print(kanto_line_list)

['東武東上線', '京浜東北線', '中央快速線', '常磐線', '小田急小田原線', '西武新宿線', '宇都宮線', '京王線', '京急本線', '西武池袋線', '東武伊勢崎線', '東海道線_(JR東日本)', '高崎線', '中央・総武緩行線', '東急田園都市線', '山手線', '東北本線', '近鉄大阪線', '京成本線', '横浜線', '横須賀線', '総武快速線', '南武線', '埼京線', '相鉄本線', '東武野田線', '東急東横線', '外房線', '新京成電鉄新京成線', '中央本線_(JR東日本)', '小田急江ノ島線', '武蔵野線', '総武本線', '都電荒川線', '東京メトロ東西線', '青梅線', '関東鉄道常総線', '成田線', '根岸線', '京王井の頭線', '都営地下鉄新宿線', '京葉線', '秩父鉄道秩父本線', '東急大井町線', '東京メトロ千代田線', '八高線', '東急池上線', '京王相模原線', '川越線', '東京メトロ日比谷線', '西武拝島線', '京成押上線', '相模線', '都営地下鉄浅草線', '江ノ島電鉄線', '東武スカイツリーライン', '東京メトロ半蔵門線', '東武アーバンパークライン', '都営地下鉄大江戸線', '東葉高速鉄道東葉高速線', '東京メトロ銀座線', '小田急多摩線', '横浜高速鉄道みなとみらい線', 'JR東西線', '首都圏新都市鉄道つくばエクスプレス', '京急空港線', '相鉄いずみ野線', '東京メトロ有楽町線', '東京臨海高速鉄道りんかい線', '鶴見線', '東京モノレール羽田空港線', '東武亀戸線', '京葉臨海鉄道臨海本線', '京王新線', '西武秩父線', '埼玉高速鉄道線', '横浜市営地下鉄ブルーライン']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
#関東(首都圏)に影響を与えそうな路線のデータのみ抽出
new_data = df_ac[df_ac['line'].isin(kanto_line_list)]
new_data = new_data.reset_index()

#不要な項目を削除
new_data = new_data.drop("index",axis=1)

new_data.head(5)

,date,time,yobi,line,from_st,to_st,st
0,2019/08/10,20:08,5,八高線,拝島駅,東福生駅,拝島駅
1,2019/08/09,22:39,4,東急東横線,学芸大学駅,学芸大学駅,学芸大学駅
2,2019/08/07,18:00,2,宇都宮線,東大宮駅,蓮田駅,東大宮駅
3,2019/08/06,11:54,1,総武本線,八街駅,日向駅,八街駅
4,2019/08/05,12:18,0,東急大井町線,自由が丘駅,九品仏駅,自由が丘駅


In [0]:
!pip install pandas-profiling

In [0]:
import pandas_profiling as pdp
from IPython.display import HTML

profile = pdp.ProfileReport(new_data)
profile.to_file(outputfile="myOutputFile_line.html")

#html化して表示
HTML(filename='myOutputFile_line.html')

# **profilingの考察**
・曜日は関係無さそう

・分析する上で情報が不足している

・正規化、細分化できる情報は無いか

In [7]:
# データフレームを定義
#通勤時間帯/帰宅時間帯/日中/それ以外で分けてみよう

#コピーを付けないとコピー元も変更されてしまう
new_df = new_data.copy()

#カラム「time」をコピーして追加
#(後でカテゴリ分けしたい為)
new_df = new_df.assign(time_category=new_df['time'])
new_df = new_df.assign(season=new_df['date'])

#-------------------------------------------------------------------------------------------
#時間帯
#-------------------------------------------------------------------------------------------
#数値型で比較したい場合は「.str[0:2].astype(int)」を使用する
new_df.loc[(new_df['time_category'].str[0:2].isin(["04","05"])) ,"time_category"] = "morning_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["06","07","08","09"])) ,"time_category"] = "comp_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["17","18","19","20","21"])) ,"time_category"] = "home_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["22","23","00","01"])) ,"time_category"] = "night_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["02","03","10","11","12","13","14","15","16"])) ,"time_category"] = "etc_time"

dmy_df = pd.get_dummies(new_df['time_category'])
add_df = pd.merge(new_df, dmy_df, left_index=True, right_index=True)
add_df = add_df.drop("time_category",axis=1)

#-------------------------------------------------------------------------------------------
#気候
#-------------------------------------------------------------------------------------------
add_df.loc[(add_df['season'].str[5:7].isin(["04","05","10","11"])) ,"season"] = "warm"
add_df.loc[(add_df['season'].str[5:7].isin(["06","07","08","09"])) ,"season"] = "hot"
add_df.loc[(add_df['season'].str[5:7].isin(["12","01","02","03"])) ,"season"] = "cold"

dmy_df = pd.get_dummies(add_df['season'])
add_df = pd.merge(add_df, dmy_df, left_index=True, right_index=True)
add_df = add_df.drop("season",axis=1)

#-------------------------------------------------------------------------------------------
#曜日
#-------------------------------------------------------------------------------------------
dmy_df = pd.get_dummies(add_df['yobi'],prefix='yobi')
add_df = pd.merge(add_df, dmy_df, left_index=True, right_index=True)
add_df = add_df.drop("yobi",axis=1)

add_df.head(5)

,date,time,line,from_st,to_st,st,comp_time,etc_time,home_time,morning_time,night_time,cold,hot,warm,yobi_0,yobi_1,yobi_2,yobi_3,yobi_4,yobi_5,yobi_6
0,2019/08/10,20:08,八高線,拝島駅,東福生駅,拝島駅,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0
1,2019/08/09,22:39,東急東横線,学芸大学駅,学芸大学駅,学芸大学駅,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0
2,2019/08/07,18:00,宇都宮線,東大宮駅,蓮田駅,東大宮駅,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
3,2019/08/06,11:54,総武本線,八街駅,日向駅,八街駅,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0
4,2019/08/05,12:18,東急大井町線,自由が丘駅,九品仏駅,自由が丘駅,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0


In [8]:
#補完したい期間の日付を入れたデータフレームを作ってマージ
#https://qiita.com/hanon/items/29cf5ed9acb4f731538f

#キー結合させる為
add_df["date"] = add_df['date'].astype('str')

temp_df = pd.read_csv("temp.csv")
temp_df = temp_df.drop("yobi", axis=1)

df_2 = pd.merge(add_df,temp_df, on='date' , how='left')#left join

df_base = df_2.copy().drop(["date","st","time","line","from_st","to_st","from_st"], axis=1)
df_base = df_base.assign(flg=1)

df_base.head()

,comp_time,etc_time,home_time,morning_time,night_time,cold,hot,warm,yobi_0,yobi_1,yobi_2,yobi_3,yobi_4,yobi_5,yobi_6,max_temp,min_temp,flg
0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,NaN,NaN,1
1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,NaN,NaN,1
2,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,NaN,NaN,1
3,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,NaN,NaN,1
4,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,NaN,NaN,1


In [9]:
import numpy as np
from datetime import date, timedelta

#日付データ作成
dates_df = pd.DataFrame({"date": [pd.to_datetime("2010-01-01") + timedelta(days=i) for i in range(3650)]},dtype="object")
dt_ary = dates_df.values.tolist()

#デフォルトでは小数点が付いてしまう為、int型に変換
time_cate = np.eye(5).astype(int)
yobi_cate = np.eye(7).astype(int)
season_cate = ['0','0','0']

wk_list = []
for wk in dt_ary:
  for wk2 in time_cate:
    for wk3 in yobi_cate:
      list_parts = np.concatenate((wk,wk2,season_cate,wk3), axis=None)
      wk_list.append(list_parts.tolist())

col=["date",
     "comp_time","etc_time","home_time","morning_time","night_time",
     "cold","hot","warm",
     "yobi_0","yobi_1","yobi_2","yobi_3","yobi_4","yobi_5","yobi_6"]

df = pd.DataFrame(data=wk_list,columns=col)

#文字列操作をする為、日付型から文字列型へ
df['date'] = df['date'].astype('str')

#-------------------------------------------------------------------------------------------
#気候
#-------------------------------------------------------------------------------------------
df.loc[(df['date'].str[5:7].isin(["04","05","10","11"])) ,"warm"] = "1"
df.loc[(df['date'].str[5:7].isin(["06","07","08","09"])) ,"hot"] = "1"
df.loc[(df['date'].str[5:7].isin(["12","01","02","03"])) ,"cold"] = "1"

temp_df = pd.read_csv("temp.csv")
temp_df = temp_df.drop("yobi", axis=1)

df = pd.merge(df,temp_df, on='date' , how='left')#left join

df_add = df.drop("date", axis=1)
df_add = df_add.assign(flg=0)
df_add.head()

,comp_time,etc_time,home_time,morning_time,night_time,cold,hot,warm,yobi_0,yobi_1,yobi_2,yobi_3,yobi_4,yobi_5,yobi_6,max_temp,min_temp,flg
0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,NaN,NaN,0
1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,NaN,NaN,0
2,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,NaN,NaN,0
3,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,NaN,NaN,0
4,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,NaN,NaN,0


In [10]:
# NegativeデータとPositiveデータを統合
ret = pd.concat([df_base, df_add], axis=0)
#duplicated()と論理否定演算子~を使って、重複した行を削除したDataFrameを取得できる。
ret = ret[~ret.duplicated()]

#中央値で埋める
ret = ret.fillna(ret.median())
ret = ret.reset_index()
ret = ret.drop("index", axis=1)

#可視化の為
ret['cold'] = ret['cold'].astype('int')
ret['warm'] = ret['warm'].astype('int')
ret['hot'] = ret['hot'].astype('int')

ret.head()

,comp_time,etc_time,home_time,morning_time,night_time,cold,hot,warm,yobi_0,yobi_1,yobi_2,yobi_3,yobi_4,yobi_5,yobi_6,max_temp,min_temp,flg
0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,15.3,8.1,1
1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,15.3,8.1,1
2,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,15.3,8.1,1
3,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,15.3,8.1,1
4,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,15.3,8.1,1


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import accuracy_score

# ラベルと入力データに分離する
y = ret.loc[:,"flg"]
y = y.values.tolist()

x = ret.loc[:,["comp_time","etc_time","home_time","morning_time","night_time","cold","hot","warm","yobi_0","yobi_1","yobi_2","yobi_3","yobi_4","yobi_5","yobi_6","max_temp","min_temp"]]

# 学習用とテスト用に分離する
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, train_size = 0.8, shuffle = True)

#グリッドサーチ
best_score = 0
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        #それぞれのパラメータの組み合わせに足してSVCを訓練
        svm = SVC(gamma=gamma, C=C)
        svm.fit(x_train, y_train)
        #SVCをテストセットで評価
        score = svm.score(x_test, y_test)
        #良いスコアであればスコアとパラメータを保持
        if score > best_score:
            best_score = score
            best_parameters = {'C':C, 'gamma':gamma}
print("Best score: {:.2f}".format(best_score))
print("Best parameters: {}".format(best_parameters))

Best score: 0.99
Best parameters: {'C': 0.001, 'gamma': 0.001}


In [12]:
# 学習する
model = SVC(C=0.001,gamma=0.001)
model.fit(x_train, y_train)

# 評価する
y_pred = model.predict(x_test)
print("正解率 = " , accuracy_score(y_test, y_pred))

正解率 =  0.9910403185664509


In [28]:
#PositiveデータとNegativeデータの割合
print(ret["flg"].value_counts())

0    9940
1     105
Name: flg, dtype: int64


In [37]:
#真陽性率 (Recall)
#Positive なデータに対してモデルがどれだけ正解できているかを確認
from sklearn.metrics import recall_score
print("真陽性率 = ",recall_score(y_test, y_pred))

真陽性率 =  0.0


###ほとんどのデータを Negative と判断していることになる･･･。

In [38]:
#適合率 (Precision) 
#モデルが Positive と判断したデータの中に、どれだけ本当に Positive なものがあったかを確認
from sklearn.metrics import precision_score
print("適合率 = ",precision_score(y_test, y_pred))

適合率 =  0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
#ダウンサンプリング
!pip install imbalanced-learn

In [41]:
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import cross_val_predict

sampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = sampler.fit_resample(x, y)

#ロジスティック回帰
clf = LogisticRegression(solver='lbfgs')
y_sampled_pred = cross_val_predict(clf, X_resampled, y_resampled, cv=5)

print("正解率 = ",accuracy_score(y_resampled, y_sampled_pred))
print("真陽性率 = ",recall_score(y_resampled, y_sampled_pred))
print("適合率 = ",precision_score(y_resampled, y_sampled_pred))

正解率 =  0.5761904761904761
真陽性率 =  0.38095238095238093
適合率 =  0.625
